<a href="https://colab.research.google.com/github/umedadada/web_scraping/blob/master/scraping_picture_motto_miru.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Colabでseleniumを使うためのセットアップ
%%shell

cat > /etc/apt/sources.list.d/debian.list <<'EOF' deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main EOF

apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517 apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138 apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

cat > /etc/apt/preferences.d/chromium.pref << 'EOF' Package: * Pin: release a=eoan Pin-Priority: 500

Package: * Pin: origin "deb.debian.org" Pin-Priority: 300

Package: chromium* Pin: origin "deb.debian.org" Pin-Priority: 700 EOF

apt-get update apt-get install chromium chromium-driver

pip install -q selenium

In [48]:
# ライブラリの読み込み
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service
import requests
import urllib.request as req

In [ ]:
# set parameta
# input
page_url="URL"
button_id="id"
# output
google_drive_save_dir = "./gdrive/My Drive/ほげほげ" # ダウンロード先の指定

In [49]:
# ブラウザをheadlessモード実行
options = webdriver.ChromeOptions()
#ヘッドレスモード（バックグラウンドで起動）で実行。コラボの場合、必須。
options.add_argument('--headless')
#サンドボックスモードの解除。これも必須。
options.add_argument('--no-sandbox')
#これも設定した方がよい。
options.add_argument('--disable-dev-shm-usage')
# 実行する場所の指定
service = Service(executable_path=r'/usr/bin/chromedriver')

In [ ]:
#インスタンス化
driver = webdriver.Chrome(service=service, options=options)

#指定したドライバーが見つかるまで待機
driver.implicitly_wait(10)

# #urlの指定
# page_url="URL"

# ブラウザの読み込み
driver.get(page_url)
time.sleep(3)

print("サイトのタイトル：", driver.title)

In [51]:
#指定したurlへ遷移する
driver.get(page_url)

#urlへ遷移する前に下の処理に行かないための記述
time.sleep(3)

In [ ]:
x = 1
while True:
    #クリックの動作を入力
    #find_element_by_idはhtmlのidの要素を指定して入力できる
    browser_from = driver.find_element(By.ID, button_id)
    time.sleep(3)
    try:
      browser_from.click()
      print('クリックしました::::'+str(x)+'回目')
      x = x + 1
    except:
      break

#指定したウェブサイトのhtmlを取得する
time.sleep(3)
parse_html = BeautifulSoup(driver.page_source, 'html.parser')

In [53]:
soup=parse_html

In [ ]:
# 画像のリンク先を取得
img_tags = soup.find_all("img")
img_urls = []

for img_tag in img_tags:
  url = img_tag.get("src")
  if url != None:
    img_urls.append(url)

# 画像ダウンロードのための関数定義
def download_image(url, file_path):
  r = requests.get(url, stream=True)
  if r.status_code == 200:
    with open(file_path, "wb") as f:
      f.write(r.content)

In [ ]:
# ダウンロード先ファイルの参照(google drive用)
import os
from google.colab import drive
drive.mount('./gdrive')
# google_drive_save_dir = "./gdrive/My Drive/ほげほげ" # ダウンロード先の指定

for index, url in enumerate(img_urls):
  file_name = "{}.jpg".format(index)
  print(file_name)
  image_path = os.path.join(google_drive_save_dir, file_name)
  print(image_path)
  try:
    download_image(url=url, file_path=image_path)
  except:
    break